In [19]:
# ** = Change location if needed
# Weight system is needed: on tags and title and for LDA to pass to LSI

In [10]:
# Seting up the environment:
# Reunite all the text file with the news in a folder called 'Analysis'
# Set up the basic path

basic_path = '/root/Desktop/' # **
path = basic_path + 'Analysis'

folders = ['/Il Corriere - Sample'] 
nick_names = ['/Corr']


choose = 0 # chose the number that represent the journal

actual_journal = [path +  folders[choose], nick_names[choose]]
print(actual_journal)

['/root/Desktop/Analysis/Il Corriere - Sample', '/Corr']


In [11]:
# Complete Analysis
# -----------------------------
# Import library

import re
import ast
import json
import nltk
import time
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [12]:
# Data loading

data = []
with open(actual_journal[0] + '.txt') as f:
    for line in f:
        data.append(json.loads(line, encoding='utf8'))
#        print(line)

        
collection_testi = []
for i in range(0, len(data)):
    if len(str(data[i]['text'])) > 10:  # Delete the news with no text
        collection_testi.append(str(data[i]['title']) + str(data[i]['text']) + str(data[i]['tags']) + str(data[i]['resume']))
        
    

In [13]:
for i in range(0, len(collection_testi)):
    f= open(actual_journal[0] + actual_journal[1] + "%s.txt" %str(i) , "w+") 
    f.write(collection_testi[i])
    f.close()


In [17]:
nltk.download("stopwords")
stop_words = set(stopwords.words('italian')) 
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
# Routine for cleaning documents:
def clean(path):
    with  open(path, 'r') as to_clean:
        one_line = ''
        for string in to_clean.readlines():
            one_line += string.lower()
    to_clean.close()
    
    # Special espressions of html format
    to_replace = ['\\n','\\t','\\r', '\\', '&nbsp'] 
    
    for item in to_replace:
        one_line = one_line.replace(item,' ') 

    cleaned = ' '.join(word for word in one_line.split() if len(word)>1)
    
    # All other special characters
    definitive = re.sub('[^a-zA-Zàéòùè]', ' ', cleaned)
    
    # Just the stopwords remain
    word_tokens = word_tokenize(definitive)
 
    filtered_text = [w for w in word_tokens if (not w in stop_words) and (w!='np')]
    cleaned = ' '.join(word for word in filtered_text if len(word)>1)
    
    with open(path, 'w') as to_clean:
        to_clean.write(cleaned) 
        
    to_clean.close()
    
    return

In [15]:
# Routine for building an interpreter of the stemmed words

def rebuild(parole_intere, parole_stemmizzate):
    vocab = dict()
    freq = dict()
    
    for i in range(0, len(parole_intere)):
        
        if parole_stemmizzate[i] not in vocab:
            vocab[parole_stemmizzate[i]] = [parole_intere[i]]
            freq[parole_intere[i]] = 1
        elif parole_intere[i] not in vocab[parole_stemmizzate[i]]:
            vocab[parole_stemmizzate[i]] =  vocab[parole_stemmizzate[i]] + [parole_intere[i]]
            freq[parole_intere[i]] = 1
        else:
            freq[parole_intere[i]] = freq[parole_intere[i]] + 1 
            
    
    return vocab, freq

In [18]:
stemmer = SnowballStemmer("italian") 
stemmed_words = []
whole_words = []

for i in range(0, len(collection_testi)):
    file_path = actual_journal[0] + actual_journal[1] + str(i)+ '.txt'
    

    cleaned_text = clean(file_path)

    full_cleaned_text = str("")

    with open(file_path, 'r') as f:
        for line in f.readlines():
            full_cleaned_text = full_cleaned_text + " " + str(line)

    tokens = nltk.word_tokenize(full_cleaned_text)
    whole_words = whole_words + tokens
    stemmed_text = ""

    for token in tokens:
        stemmed_text = stemmed_text + " " + stemmer.stem(token)
        stemmed_words.append(stemmer.stem(token))
    with open(file_path, 'w') as f:
        f.write(stemmed_text)


In [19]:
interpreter, counter = rebuild(whole_words, stemmed_words)
# print(interpreter)

In [20]:
import findspark
findspark.init("/usr/local/spark") # **
import pyspark
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [21]:
from pyspark.sql import SQLContext, Row
from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.ml.feature import StopWordsRemover

lista_topic = []
lista_pesi = []

#------ STARTING LDA PROCEDURE ----------
start_LDA_time = time.time()

for news in range(0, len(collection_testi)): 
    
    file_path = actual_journal[0] + actual_journal[1] + str(news)+ '.txt' 

    data = sc.textFile(file_path).zipWithIndex().map( lambda cleanwords_idd: Row(idd = cleanwords_idd[1], cleanwords = cleanwords_idd[0].split(" ")))
    docDF = spark.createDataFrame(data)


    Vector = CountVectorizer(inputCol="cleanwords", outputCol="vectors")
    model = Vector.fit(docDF)
    result = model.transform(docDF)

    corpus = result.select("idd", "vectors").rdd.map(lambda x_y: [x_y[0],Vectors.fromML(x_y[1])]).cache()

    # Cluster the documents into k topics using LDA
    ldaModel = LDA.train(corpus, k = 2, maxIterations = 100, optimizer='online') # em
    topics = ldaModel.topicsMatrix()
    vocabArray = model.vocabulary

    weight = ldaModel.describeTopics(maxTermsPerTopic = 12) # Pay attention to the number of maxTermsPerTopic 
                                                            # it must be consistent

    wordNumbers = 12  # number of words per topic
    topicIndices = sc.parallelize(ldaModel.describeTopics(maxTermsPerTopic = wordNumbers))
    
    def topic_render(topic):  # specify vector id of words to actual words
        terms = topic[0]
        result = []
        for i in range(wordNumbers):
            term = vocabArray[terms[i]]
            result.append(term)
        return result
    
    
    topics_final = []
    topics_final = topicIndices.map(lambda topic: topic_render(topic)).collect()
    
    i = 0 
    print('---------- NEWS n° %s' %news)    
    for topic in range(len(topics_final)):
        print ("Topic" + str(topic) + ":")
        j = 0
        terms = ''
        for term in topics_final[topic]:
            print (term + '  ' + str(weight[i][1][j]))
            terms = terms + ' ' + term
            lista_pesi.append(weight[i][1][j])
            j += 1
        i += 1
        print ('\n')
        lista_topic.append(terms) 
        
print("--- The LDA procedure took  %s seconds  ---" % (time.time() - start_LDA_time))

---------- NEWS n° 0
Topic0:
pot  0.006328716055309242
fam  0.006208742242230946
nient  0.006129731723750403
vol  0.006072223383553896
dop  0.006045901254260223
dov  0.005997727499597295
mort  0.005970311461602011
altri  0.005848417754195531
omicid  0.005834498687304169
capac  0.005831271075780091
colpevol  0.005798292507635383
mal  0.0057975575232233865


Topic1:
riin  0.00668553930986834
cos  0.006588499977436611
bagarell  0.0063419405585640656
maf  0.006044750042755046
dir  0.006021733793793081
ignor  0.005836690221697481
cap  0.00580026492044782
capitol  0.0057961138009314074
lett  0.005787426573563398
ginocc  0.005741121847403726
anni  0.005707630392235834
pens  0.005680250604422833


---------- NEWS n° 1
Topic0:
imam  0.006630665485775352
nacional  0.006604282130613948
stat  0.0059940546654387965
barcellon  0.00592229588540704
es  0.005863673243011137
rambl  0.005858876612972064
cni  0.005772762011057636
abouyaakoub  0.005680620889591882
terror  0.005667393970588512
agent  0.0056

---------- NEWS n° 12
Topic0:
sottomarin  0.007565199915765102
problem  0.006284026205455222
san  0.0060816139180759335
nè  0.006002176930970517
argentin  0.005926520966318171
nav  0.005869339688421538
poss  0.005697055706170847
raccont  0.005658999144291947
convenzional  0.005592847669910123
oltre  0.005587703408136526
ancor  0.005577542695218741
altri  0.005552393799506232


Topic1:
frequenz  0.006724315160463365
compliment  0.005947557772947331
problem  0.005856167724797652
soltant  0.005825283391935666
mezz  0.0057843691262347684
portavoc  0.005697383909752708
punt  0.005690648241026487
antenn  0.005651568223285269
aires  0.0056408856783567935
domand  0.005605036493676374
bas  0.005593693042104498
dett  0.0055905695838510995


---------- NEWS n° 13
Topic0:
arma  0.010553493100414835
cittadin  0.010351669313585655
special  0.010339042576646624
paes  0.0102615067414504
possed  0.009870818931842582
adott  0.0098168290966946
cos  0.009799278294361476
scend  0.009723640177161624
animal 

---------- NEWS n° 24
Topic0:
vett  0.01895109152426414
manten  0.01815327665733544
dev  0.017877547763224512
pallon  0.017542676752732907
scudett  0.01729191176321833
entramb  0.017230801799296588
vint  0.017152848075077075
europ  0.016937247735031042
vinc  0.016776517035964722
sin  0.016772504155768458
centrocamp  0.0167320434530572
spicc  0.01658240907141098


Topic1:
derby  0.019902066006651495
protagon  0.01904257825783331
laz  0.018285269331756546
vett  0.01767049317577544
duemil  0.017515034455279268
nov  0.01722935345422143
rom  0.017211269131647184
prov  0.01712584690871852
entramb  0.016672424201895412
elimin  0.016621368248991637
dev  0.01637617261225812
ultim  0.016238610422024624


---------- NEWS n° 25
Topic0:
vittim  0.012754658839547945
ragazz  0.012694549030356229
prim  0.012487206599897013
tend  0.012094157081447598
abbi  0.012032184249470489
scardin  0.011966842495382416
facil  0.011800360981963187
respint  0.01164449688139707
esser  0.01150870046816316
incub  0.0114

---------- NEWS n° 36
Topic0:
melegatt  0.008528291242186686
arriv  0.008186287173712026
produzion  0.008015584219656527
fond  0.007630940505358731
chairman  0.007529382060012362
stat  0.007387520719688927
pandor  0.0072010645261371825
commissar  0.007130539247745587
futur  0.007104669111109449
ricominc  0.007087836179345688
graz  0.0070843016589124345
giorn  0.007083390730762879


Topic1:
cris  0.007672246625986419
lavor  0.0075716549540670655
stip  0.007450374002580059
stor  0.007358154607936858
aggiunt  0.007339885756261175
quagin  0.007323003324235619
tribunal  0.007278344429012384
sosten  0.007264108700578152
provenient  0.0072005331264643115
prev  0.007166162200683294
qual  0.007163078628505686
stess  0.007158164837858123


---------- NEWS n° 37
Topic0:
support  0.0066668757731303385
affinc  0.006325770994575237
consent  0.0063163873538619126
ordin  0.006227622873318276
sempr  0.006215979709424726
massim  0.006180807577897548
mentr  0.006172810028318638
merc  0.006167929472399366

---------- NEWS n° 48
Topic0:
far  0.017086060289739854
compagn  0.013742750244298331
vogl  0.013227049168272938
alital  0.01169840748441211
lufthans  0.01136219263790879
ministr  0.01125418953325178
delr  0.010963620804556715
vol  0.01080507069056015
situazion  0.010713734190660115
commissar  0.010710576484924784
afric  0.010639111531068998
tedesc  0.010581005802486033


Topic1:
vol  0.010446743311723353
  0.00978901884845379
spieg  0.009703315498631576
trasport  0.009662487909294205
tropp  0.009627648913361823
met  0.009612103290000482
cit  0.009493094880430222
alital  0.009467527542878329
compagn  0.009374439785166524
infatt  0.009289379448268862
tedesc  0.009226464614247072
success  0.009196310923712704


---------- NEWS n° 49
Topic0:
dop  0.008922098571590343
anni  0.008384356727944381
mort  0.008289088320534424
oral  0.008153586727026157
sent  0.008116376956819249
lung  0.008009505365445577
ann  0.008008217272408429
prim  0.008005827043122811
formal  0.007974036942864306
sylveste

...

---------- NEWS n° 684
Topic0:
ognun  0.009288217495960474
vest  0.009187070705714437
strad  0.009063946552638385
fil  0.00903511587501651
amat  0.009013016682726244
meravigl  0.00900141265347077
parl  0.00888375417969442
so  0.008781201399957387
squadr  0.0087453225876121
uso  0.008702066024894718
vit  0.008619421120204102
anni  0.00850645810566069


Topic1:
fa  0.00938567129893562
vit  0.008992438061233655
avvicin  0.008734211256435713
qual  0.008710330433235621
innov  0.008702926201088394
italian  0.008611919935258033
paes  0.008606131925193324
squadr  0.008551099628048783
vogl  0.008467702263997973
person  0.008391885135796586
figl  0.008346977151882179
pettin  0.008342396578714948


---------- NEWS n° 685
Topic0:
libr  0.0114039948358767
scrittor  0.009579088087683218
classif  0.009137960812269636
lettur  0.009009809794106906
traduzion  0.008855175241934738
traduttor  0.008614947034443413
second  0.008560512785728079
oper  0.008535521702085278
vot  0.008390812804281349
anno  0.008

---------- NEWS n° 696
Topic0:
illegal  0.014212760335664628
impost  0.013873287587645567
apert  0.013714509232747642
tutt  0.01362770115705509
sistem  0.013590511244721054
protett  0.01303478919385186
ricev  0.012930668174633678
pot  0.012646690246716866
iniz  0.012625396125943818
virtual  0.012569329030490392
fiduc  0.012534395407097857
mod  0.012394030035614617


Topic1:
bitcoin  0.018471912720898126
monet  0.015378533889429776
controll  0.014437478810334203
senz  0.01438666788183874
transazion  0.014338326881228124
scamb  0.013713762915696103
partecip  0.013647623544729816
possibil  0.013326663458282297
riconosc  0.01332625699418187
mod  0.013097101603680154
fatt  0.013086421965498666
sistem  0.012767675137892209


---------- NEWS n° 697
Topic0:
quotidian  0.009030743679510836
bett  0.00882837030443223
anni  0.008666055467808229
passegger  0.008608965795189048
hostess  0.008555932330241474
vol  0.008394539339949391
signor  0.008260207571115967
rispond  0.008122566954715221
ved  0.0

---------- NEWS n° 708
Topic0:
diamant  0.004529854046713828
cener  0.004526677464471663
anni  0.004157580013041308
poi  0.004071635826213054
car  0.003958229209167924
mort  0.003907385664287285
dop  0.0038773685204183195
sembr  0.003861475237128394
client  0.003795420020280146
cos  0.003782510546682487
gramm  0.0037613089311332236
gioiell  0.003739451475947034


Topic1:
fisic  0.004032539302681333
element  0.0038505382125222087
graf  0.003823208412944531
abbandon  0.003811563782114706
terrestr  0.0037575051935764894
ital  0.00374243793373544
ogni  0.003661948506011742
darl  0.003660726420384508
butt  0.0036540665618172725
confront  0.0036370665765246457
person  0.0036268708362080498
malincon  0.0036266997085592197


---------- NEWS n° 709
Topic0:
leopard  0.011090923088425348
grand  0.010442718563195258
pessim  0.008004131205204082
portabors  0.007727780033626306
manzon  0.007695668991428415
professor  0.007676449236694295
altro  0.007675727035117072
sol  0.007570777155085623
appunt  

---------- NEWS n° 720
Topic0:
centrist  0.026947813152679134
miglior  0.024758662782721878
democraz  0.02450265622337232
dilemm  0.023629406492237017
attanagl  0.023359218769603042
europ  0.023060359848671785
udc  0.02300878402099317
popolar  0.02270835277729399
vann  0.02212470509612475
civic  0.02206896778445788
autonom  0.02201210802927717
vist  0.021845283028364748


Topic1:
solidal  0.02709959637578164
strateg  0.026186123798483715
ferdin  0.02507508276817789
autonom  0.02409463272349111
popol  0.023651073008048532
direzion  0.023583026199528644
grand  0.02349369313182672
different  0.023401211052076373
gall  0.023380575634678692
moltepl  0.02288842103927492
sfumatur  0.0224319191056322
popolar  0.022012142672222325


---------- NEWS n° 721
Topic0:
impegnin  0.00433536287305011
palestines  0.00433064314612775
bruc  0.004316979955343622
rispond  0.004285910150341677
ebre  0.004240434842108067
poi  0.004205388215742962
onu  0.0041866917628177605
port  0.004135794795869937
premier  

---------- NEWS n° 732
Topic0:
effett  0.013335275791356913
habb  0.01272776792822266
italian  0.01259382420843061
onor  0.012585432913248668
sent  0.012410791906955326
gall  0.012327055492282137
cadrebber  0.01222295136054491
infin  0.012158146543863789
stor  0.012118715057227092
sbarc  0.011987599124274311
cant  0.01187162320187005
bomb  0.011845468623053733


Topic1:
mut  0.012982620391010008
divers  0.012958235260951037
allor  0.012759774307482251
sbarc  0.012564191521016073
bicchier  0.012334557249823372
pronunc  0.012284725005158006
scuol  0.012076950430888812
effett  0.012013546839129493
rodar  0.011927041743804684
vist  0.011915050891153767
cum  0.011902107224452157
restitu  0.011883048195707449


---------- NEWS n° 733
Topic0:
nient  0.006333764286064656
autorizz  0.006323660826789689
tutt  0.006296324483859016
organizz  0.005910812962329511
indagin  0.005883762060388242
mifsud  0.005855813091301296
person  0.0058346675896436065
interess  0.005812878940716966
dic  0.0057606025

In [46]:
# Clean again the words... 740 articoli per 2194 sec

more_stopwords = ['me', 'poi', 'fin', 'dop', 'quand', 'aver', 'senz', 'cos', 'due', 'molt', 'nuov', 'fa', 'esser',
                  'sol', 'mai', 'può', 'prim', 'tant', 'second', 'grand', 'fin', 'via', 'piccol', 'ora', 'sempr', 
                  'tre', 'dic', 'ancor', 'propr', 'mes', 'anno', 'anni', 'scors', 'far', 'arriv', 'sembr', 'circ'
                 'port', 'temp', 'post', 'oltre', 'stess', 'tutt', 'volt',  'po', 
                  'mentr', 'altri', 'invec', 'fatt', 'giorn', 'cas', 'oggi',] 

# Time, size, auxiliar verbs... Extra words 'fatt', 'giorn', 'cas', 'oggi']
    
    

for i in range(0, len(lista_topic)):
    
    word_tokens = word_tokenize(lista_topic[i])
    filtered_text = [w for w in word_tokens if not w in more_stopwords]
    lista_topic[i] = ' '.join(word for word in filtered_text)
    
print(lista_topic)

['pot fam nient vol dov mort altri omicid capac colpevol mal', 'riin bagarell maf dir ignor cap capitol lett ginocc pens', 'imam nacional stat barcellon es rambl cni abouyaakoub terror agent stralc', 'stralc quattr nitroglicerin quind contatt spagnol comunqu ambient elimin organizz enne', 'graz altra unveiling musk californ jolly fors camion colp batter', 'liquid obblig californ spost distrarr tecnolog nessun cart prestazion inventor', 'ner louis linton pell tesor domin fiss lung web mnuchin', 'maggior gang tesor compars vest iron diciottenn cert guant risult ultim comment', 'direttor espert omonim stagion creat lung estrazion per ball color persin strizz', 'cappott divent stamp color conviv stil fior abit oggett cap', 'napol laz assest potrebb part rest sembr vicin chiar', 'inter vicin napol sampdor sorpres med belott favor sembr', 'ier campagn ospit import arterioscler uscit elisabett teres spieg autor', 'raccont stor lettur numer sal novembr ciabatt autor omagg libr parol', 'vist ca

In [49]:
vectorizer = TfidfVectorizer() # use_idf=True, ngram_range=(1, 3)

number_topic =  20
number_words = 15

#------ STARTING LSI PROCEDURE ----------
start_LSI_time = time.time()

X = vectorizer.fit_transform(lista_topic) # with range we can capture latent structure 

lsa = TruncatedSVD(n_components = number_topic, n_iter = 1000) # Gauge the components
lsa.fit(X)

terms = vectorizer.get_feature_names()
out = []
for i, comp in enumerate(lsa.components_):
    termsInComp = zip(terms,comp)
    sortedTerms = sorted(termsInComp, key=lambda x: x[1], reverse=True ) [:number_words]
    
    for term in sortedTerms:
        out.append(term[0])
        
print("--- The LSI procedure took  %s seconds  ---" % (time.time() - start_LSI_time))

# We could sum and/or compare times at this point...

--- The LSI procedure took  10.363251209259033 seconds  ---


In [50]:

index_topic = 1
index_words = 1

print('Topic %s' %index_topic )

for w in out:

    print(str(index_words) + ') ' + w)
    occur = []
    for i in range(0, len(interpreter[w])):
        occur = occur + [[interpreter[w][i]] + [counter[interpreter[w][i]]]] 
    
    occur.sort(key=lambda x: x[1], reverse = True)
    print(occur)
    print('   ')
    
    if index_words == number_words:
        index_topic += 1
        print('Topic %s' %index_topic)
        index_words = 1
    else:
        index_words +=1
    

Topic 1
1) stat
[['stato', 752], ['stata', 419], ['stati', 257], ['state', 98], ['statuto', 3]]
   
2) part
[['parte', 366], ['partito', 95], ['partire', 57], ['partita', 53], ['parti', 33], ['partiti', 24], ['partite', 22], ['partendo', 8], ['parto', 6], ['partono', 4], ['partirà', 3], ['partiva', 2], ['partirono', 2], ['partivano', 2], ['partirebbe', 1], ['parta', 1], ['partano', 1]]
   
3) ital
[['italia', 376], ['italiano', 104], ['italo', 4], ['italici', 2], ['italico', 2], ['italiche', 1]]
   
4) lavor
[['lavoro', 301], ['lavoratori', 59], ['lavorare', 48], ['lavori', 43], ['lavorato', 36], ['lavora', 34], ['lavorando', 20], ['lavorano', 15], ['lavorava', 10], ['lavoravano', 4], ['lavoratrici', 4], ['lavorativo', 3], ['lavoratore', 3], ['lavoratrice', 3], ['lavorati', 3], ['lavoravo', 2], ['lavorativa', 2], ['lavorazione', 2], ['lavorative', 1], ['lavoriamo', 1], ['lavorerà', 1], ['lavorò', 1]]
   
5) mil
[['milano', 253], ['mila', 210], ['milita', 4]]
   
6) polit
[['politica', 

[['serie', 114], ['sera', 80], ['serata', 30], ['serio', 20], ['seria', 12], ['seriamente', 7], ['seri', 4], ['serate', 3], ['sere', 1]]
   
10) uom
[['uomo', 184]]
   
11) accus
[['accusa', 72], ['accuse', 41], ['accusato', 38], ['accusata', 11], ['accusati', 8], ['accusate', 4], ['accusò', 3], ['accusano', 3], ['accusarono', 2], ['accusandola', 2]]
   
12) mil
[['milano', 253], ['mila', 210], ['milita', 4]]
   
13) general
[['generale', 110], ['generali', 10], ['general', 5], ['generalista', 3], ['generalmente', 2], ['generaliste', 1]]
   
14) lavor
[['lavoro', 301], ['lavoratori', 59], ['lavorare', 48], ['lavori', 43], ['lavorato', 36], ['lavora', 34], ['lavorando', 20], ['lavorano', 15], ['lavorava', 10], ['lavoravano', 4], ['lavoratrici', 4], ['lavorativo', 3], ['lavoratore', 3], ['lavoratrice', 3], ['lavorati', 3], ['lavoravo', 2], ['lavorativa', 2], ['lavorazione', 2], ['lavorative', 1], ['lavoriamo', 1], ['lavorerà', 1], ['lavorò', 1]]
   
15) gallitell
[['gallitelli', 16]]
   

In [107]:
#  Saving
with open(path + actual_journal[1] + ' Output', "w") as output:
    output.write("\n".join(map(lambda x: str(x), out)))